In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from pylab import rcParams
from IPython.display import display
import math

import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
# pd.set_option('future.no_silent_downcasting', False)


#### Data preparation

In [18]:
# open and read excel file
df = pd.read_excel("../input data/Данные-рус Обработанные (новые).xlsx")
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
2,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
3,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
4,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69


In [19]:
# remove the first row (blank)
df = df.drop(index=df.index[:1]).reset_index(drop=True)
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122
0,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
1,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
2,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
3,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
4,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,...,1055.43,110.45,188.15,172.63,112.31,134.99,1212.02,260.02,129.38,159.66


In [20]:
# Set the first row as column labels
df.columns = df.iloc[0]  # Assign first row as header
df = df[1:].reset_index(drop=True)  # Drop the first row and reset index
df.head(5)

,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
0,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
1,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
2,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
3,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,...,1055.43,110.45,188.15,172.63,112.31,134.99,1212.02,260.02,129.38,159.66
4,КРС,Сельхозпредприятия,АТЫРАУСКАЯ ОБЛАСТЬ,20.8,0.8,15.28,1.97,2.04,1.32,0.8,...,13.52,25.1,13.7,16.08,17.35,41.57,56.59,49.5,40.76,27.22


In [21]:
# convert date columns to datetime format
df.columns = list(df.columns[:3]) + list(pd.to_datetime(df.columns[-120:]))
df = df.rename(columns={col: col.strftime("%Y-%m") for col in df.columns[-120:]}) # change "2015-01-01 00:00:00" to "2015-01" 
df.head(5)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
1,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
2,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
3,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,...,1055.43,110.45,188.15,172.63,112.31,134.99,1212.02,260.02,129.38,159.66
4,КРС,Сельхозпредприятия,АТЫРАУСКАЯ ОБЛАСТЬ,20.8,0.8,15.28,1.97,2.04,1.32,0.8,...,13.52,25.1,13.7,16.08,17.35,41.57,56.59,49.5,40.76,27.22


In [22]:
# check values
print(df["Показатель"].unique())
print(df["Категория хозяйства"].unique())
print(df["Регион"].unique())

['КРС' 'Свиньи' 'Лошади' 'Верблюды' 'Птица' 'Молоко' 'Яйца' 'Овцы и козы']
['Сельхозпредприятия' 'Крестьянские или фермерские хозяйства'
 'Хозяйства населения']
['РЕСПУБЛИКА КАЗАХСТАН' '    АКМОЛИНСКАЯ ОБЛАСТЬ'
 '    АКТЮБИНСКАЯ ОБЛАСТЬ' '    АЛМАТИНСКАЯ ОБЛАСТЬ'
 '    АТЫРАУСКАЯ ОБЛАСТЬ' '    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ЖАМБЫЛСКАЯ ОБЛАСТЬ' '    КАРАГАНДИНСКАЯ ОБЛАСТЬ'
 '    КОСТАНАЙСКАЯ ОБЛАСТЬ' '    КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ'
 '    МАНГИСТАУСКАЯ ОБЛАСТЬ' '    ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ПАВЛОДАРСКАЯ ОБЛАСТЬ' '    СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ' '    ГАСТАНА' '    ГАЛМАТЫ'
 '    ГШЫМКЕНТ' '    ТУРКЕСТАНСКАЯ ОБЛАСТЬ' '    ОБЛАСТЬ ЖЕТІСУ'
 '    ОБЛАСТЬ ҰЛЫТАУ' '    ОБЛАСТЬ АБАЙ']


In [23]:
df["Регион"] = df["Регион"].str.strip() # strip unnecessary spaces
df = df[df["Регион"] != "ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ"] # remove ЮКО
df["Регион"].unique()

array(['РЕСПУБЛИКА КАЗАХСТАН', 'АКМОЛИНСКАЯ ОБЛАСТЬ',
       'АКТЮБИНСКАЯ ОБЛАСТЬ', 'АЛМАТИНСКАЯ ОБЛАСТЬ', 'АТЫРАУСКАЯ ОБЛАСТЬ',
       'ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ЖАМБЫЛСКАЯ ОБЛАСТЬ',
       'КАРАГАНДИНСКАЯ ОБЛАСТЬ', 'КОСТАНАЙСКАЯ ОБЛАСТЬ',
       'КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ', 'МАНГИСТАУСКАЯ ОБЛАСТЬ',
       'ПАВЛОДАРСКАЯ ОБЛАСТЬ', 'СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ',
       'ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ГАСТАНА', 'ГАЛМАТЫ', 'ГШЫМКЕНТ',
       'ТУРКЕСТАНСКАЯ ОБЛАСТЬ', 'ОБЛАСТЬ ЖЕТІСУ', 'ОБЛАСТЬ ҰЛЫТАУ',
       'ОБЛАСТЬ АБАЙ'], dtype=object)

In [25]:
# fill missing values with 0
df = df.fillna(0)
# replace "-" with 0
df.replace("-", 0, inplace=True)
df.tail(5)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
523,Овцы и козы,Хозяйства населения,ГШЫМКЕНТ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,54.6,43.7,46.4,44.2,42.0,39.7,50.7,49.0,46.2,174.8
524,Овцы и козы,Хозяйства населения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3904.3,3414.9,3224.1,3182.1,1823.1,2272.9,3341.6,3790.5,4074.5,5796.7
525,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ЖЕТІСУ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2077.3,656.4,500.8,3064.5,547.4,1156.3,2134.4,2237.4,2093.4,3738.4
526,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ҰЛЫТАУ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,198.3,65.4,490.1,376.7,86.4,213.8,455.5,216.7,169.2,216.1
527,Овцы и козы,Хозяйства населения,ОБЛАСТЬ АБАЙ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,509.0,638.0,659.1,1385.8,439.4,550.8,741.9,476.9,601.3,2328.2


In [ ]:
# Step 1: Define target regions and periods
regions_to_null = ['ОБЛАСТЬ ЖЕТІСУ', 'ОБЛАСТЬ ҰЛЫТАУ', 'ОБЛАСТЬ АБАЙ']
periods_to_null = pd.date_range("2015-01", "2022-05", freq="MS").strftime("%Y-%m")

# Step 2: Replace matching values with NaN
mask = (
    df["Регион"].isin(regions_to_null) &
    df.columns[3:].to_series().isin(periods_to_null).any()
)

# Step 3: Set values to NaN across all columns for those rows and columns
value_columns = df.columns[3:]  # assuming first 3 columns are identifiers

for period in periods_to_null:
    if period in value_columns:
        df.loc[df["Регион"].isin(regions_to_null), period] = np.nan
df.tail(5)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
523,Овцы и козы,Хозяйства населения,ГШЫМКЕНТ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,54.6,43.7,46.4,44.2,42.0,39.7,50.7,49.0,46.2,174.8
524,Овцы и козы,Хозяйства населения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3904.3,3414.9,3224.1,3182.1,1823.1,2272.9,3341.6,3790.5,4074.5,5796.7
525,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ЖЕТІСУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2077.3,656.4,500.8,3064.5,547.4,1156.3,2134.4,2237.4,2093.4,3738.4
526,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ҰЛЫТАУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,198.3,65.4,490.1,376.7,86.4,213.8,455.5,216.7,169.2,216.1
527,Овцы и козы,Хозяйства населения,ОБЛАСТЬ АБАЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,509.0,638.0,659.1,1385.8,439.4,550.8,741.9,476.9,601.3,2328.2


In [28]:
# Step 1: Define target regions and periods
regions_to_null2 = ['ГШЫМКЕНТ', 'ТУРКЕСТАНСКАЯ ОБЛАСТЬ']
periods_to_null2 = pd.date_range("2015-01", "2018-05", freq="MS").strftime("%Y-%m")

# Step 2: Replace matching values with NaN
mask = (
    df["Регион"].isin(regions_to_null2) &
    df.columns[3:].to_series().isin(periods_to_null2).any()
)

# Step 3: Set values to NaN across all columns for those rows and columns
value_columns = df.columns[3:]  # assuming first 3 columns are identifiers

for period in periods_to_null2:
    if period in value_columns:
        df.loc[df["Регион"].isin(regions_to_null2), period] = np.nan
df.tail(5)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
523,Овцы и козы,Хозяйства населения,ГШЫМКЕНТ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.6,43.7,46.4,44.2,42.0,39.7,50.7,49.0,46.2,174.8
524,Овцы и козы,Хозяйства населения,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3904.3,3414.9,3224.1,3182.1,1823.1,2272.9,3341.6,3790.5,4074.5,5796.7
525,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ЖЕТІСУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2077.3,656.4,500.8,3064.5,547.4,1156.3,2134.4,2237.4,2093.4,3738.4
526,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ҰЛЫТАУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,198.3,65.4,490.1,376.7,86.4,213.8,455.5,216.7,169.2,216.1
527,Овцы и козы,Хозяйства населения,ОБЛАСТЬ АБАЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,509.0,638.0,659.1,1385.8,439.4,550.8,741.9,476.9,601.3,2328.2


In [38]:
# check replacement
print(df.groupby("Регион")[df.columns[90:94]].apply(lambda g: g.isna().sum()).reset_index())
print(df.groupby("Регион")[df.columns[42:46]].apply(lambda g: g.isna().sum()).reset_index())

                            Регион  2022-04  2022-05  2022-06  2022-07
0              АКМОЛИНСКАЯ ОБЛАСТЬ        0        0        0        0
1              АКТЮБИНСКАЯ ОБЛАСТЬ        0        0        0        0
2              АЛМАТИНСКАЯ ОБЛАСТЬ        0        0        0        0
3               АТЫРАУСКАЯ ОБЛАСТЬ        0        0        0        0
4   ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ        0        0        0        0
5                          ГАЛМАТЫ        0        0        0        0
6                          ГАСТАНА        0        0        0        0
7                         ГШЫМКЕНТ        0        0        0        0
8               ЖАМБЫЛСКАЯ ОБЛАСТЬ        0        0        0        0
9    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ        0        0        0        0
10          КАРАГАНДИНСКАЯ ОБЛАСТЬ        0        0        0        0
11            КОСТАНАЙСКАЯ ОБЛАСТЬ        0        0        0        0
12          КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ        0        0        0        0
13    

### 

In [39]:
# get the final data by summing across показатели & категория хозяйства
totals = df.groupby(["Показатель", "Регион"])[df.columns[3:]].agg(lambda x: x.sum() if x.notna().any() else np.nan).reset_index()
totals

C:\Users\mjaks\AppData\Local\Temp\ipykernel_11424\3052080860.py:2: FutureWarning: Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.
  totals = df.groupby(["Показатель", "Регион"])[df.columns[3:]].agg(lambda x: x.sum() if x.notna().any() else np.nan).reset_index()


,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,...,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,...,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,...,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,...,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70,4.20,1.80,...,0.0,0.8,0.00,0.0,0.50,0.0,0.00,0.40,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Яйца,ОБЛАСТЬ ҰЛЫТАУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1569.2,1156.4,1174.80,1957.4,1310.40,973.5,1757.90,961.60,806.3,1780.60
164,Яйца,ПАВЛОДАРСКАЯ ОБЛАСТЬ,9637.30,8361.40,9164.50,10447.60,14083.60,14200.20,14200.20,12827.60,...,16631.1,16747.1,17414.00,18832.4,18654.70,18689.8,18071.90,16535.50,15899.2,14834.90
165,Яйца,РЕСПУБЛИКА КАЗАХСТАН,325795.20,303234.00,376664.50,400243.30,462679.40,454045.70,454045.70,431318.00,...,381480.5,368147.0,377486.90,393946.9,397493.70,394871.5,385885.00,371208.00,362418.8,364206.50
166,Яйца,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,37704.50,37570.60,41732.40,52925.70,68018.40,79389.60,79389.60,57759.90,...,49356.3,49099.2,51500.30,56233.7,58261.10,53762.5,51783.90,47959.40,47352.0,48441.50


### Concat with other indicators dataset (температура, осадки, поголовье)

In [ ]:
x_vars = pd.read_excel("C:/Users/Acer/Desktop/Madi/Tasks/Nowcasting/Data/Data cleansing/output data/доп регрессоры обработанные.xlsx")
x_vars.head(5)

In [ ]:
df_combined = pd.concat([totals, x_vars], ignore_index=True)
df_combined.sample(10)

In [ ]:
# save into an excel file: все 8 показателей в сумме по категориям хозяйств в одном excel
df_combined.to_excel("output data/Просуммированные по категориям с доп регрессорами.xlsx", index=False)

In [ ]:
# # save into excel files: 8 показателей в сумме по категориям хозяйств, каждый в отдельном excel

# indicators = ["КРС", "Лошади", "Свиньи", "Овцы и козы", "Птицы", "Молоко", "Яйца", "Верблюды"]

# for indicator in indicators:
#     df_filtered = df_combined[df_combined["Показатель"] == indicator].drop(columns=["Показатель"])
#     file_name = f"output data/{indicator}_обработанные.xlsx"
#     df_filtered.to_excel(file_name, index=False)
